Downloading the required libraries

In [1]:
!pip install langchain  
!pip install openai
!pip install python-dotenv
!pip install faiss-cpu
!pip install tiktoken
!pip install google-search-results #change to duckduckgo when no token uwu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 72.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32002 sha256=685faf900ace9f1a63a88636040478eabbc971f14992c20b82fb9c74443e0c80
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


Importing the required libraries

OS: Used for setting environment variables (OpenAI key)
langchain.document_loaders: For loading documents 
langchain.prompts: For prompt templates
langchain.text_splitter: For chunking of documents 
langchain.embeddings: For accessing openai embeddings used for converting document text into embedding vectors 
FAISS: vectorstore to store embeddings
langchain.chains: To import different kinds of chains which is the basic functional block of langchain
langchain.agents: To import different kinds of agents which can combine llm chains and tools
langchain.schema: To import functions required for creating custom agents (which have custom system prompts)

-- I dont remember where and why I used list and union- perhaps for llm output text parsing 

-- re is regular expression library- once again for llm out text parsing / string manipulation 

In [2]:
import os
from langchain.document_loaders import UnstructuredFileLoader 
from langchain.prompts import PromptTemplate,StringPromptTemplate,StringPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
import pickle
from langchain.chains import RetrievalQA, LLMChain
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser, AgentType
from langchain.schema import SystemMessage,AgentAction, AgentFinish, OutputParserException
from typing import List, Union
import re

Loading the openai key below. With this we can access openai models. Alternatives exist like open source huggingface models such as LLama and Falcon7B

Openai models like Davinci and GPT3.5-turbo show much better performance in comparison to these open source models

Enter your own openai key to run the code. 

In [12]:
os.environ["OPENAI_API_KEY"] = ''

Unstructured library is required for the manipulation and loading of documents, without which an error for the same will occur. 

In [13]:
!pip install unstructured==0.7.12


Set openai embeddings 

In [14]:
embeddings = OpenAIEmbeddings()

In the following code cells, we construct the vectorstores corresponding to different documents- Indian Penal Code, Consumer Protection Act 2019 etc. We load the document using a loader, split it into chunks, and then convert the text into embedding vectors and store in a vectorstore.

In [15]:
law_fields_loader = UnstructuredFileLoader('/kaggle/input/law-fields/LawFields.pdf')
docs = law_fields_loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)

documents = text_splitter.split_documents(docs)

law_fields_vectorstore = FAISS.from_documents(documents, embeddings)

with open("law_fields_vectorstore.pkl", "wb") as f:
    pickle.dump(law_fields_vectorstore, f)

In [16]:
indian_penal_code_loader = UnstructuredFileLoader('/kaggle/input/consumercourt/ConsumerCourt.pdf')
docs = indian_penal_code_loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)

documents = text_splitter.split_documents(docs)

indian_penal_code_vectorstore = FAISS.from_documents(documents, embeddings)

with open("indian_penal_code_vectorstore.pkl", "wb") as f:
    pickle.dump(indian_penal_code_vectorstore, f)

In [17]:
consumer_protection_act_loader = UnstructuredFileLoader('/kaggle/input/consumerprotectionact2019/consumer_protection_act_2019.pdf')
docs = consumer_protection_act_loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)

documents = text_splitter.split_documents(docs)

consumer_protection_act_vectorstore = FAISS.from_documents(documents, embeddings)

with open("consumer_protection_act_vectorstore.pkl", "wb") as f:
    pickle.dump(consumer_protection_act_vectorstore, f)

langchain.chatmodels: To import OpenAI models- here we use gpt3.5-turbo

In [18]:
from langchain.chat_models import ChatOpenAI
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

The LawType() function takes in a user query and outputs one of 18 different law types to which the particular query relates most with. 

For this purpose a RetrievalQnA chain is used. It uses a retreiver that has information from the law_fields document. This document has information on 18 types of law.

In [19]:
def LawType(query):
    
    #prompt for classifying into types
    law_type_template = """
    You are an assistant who is an expert in understanding law and interpreting language related to law. The user has a problem
    which they describe in a situation. You are to use the information context given to you to determine which type of law their
    problem falls under. 
    
    The output must ONLY be the name of law type that it is most likely to fall under.
    
    Mention only one law type.
    
    Hence outputs will most often be 2-3 words long and always be less than 5 words long. Do not say anything else.
    
    The information context: {context}

    Do NOT assume more information than what the situation and ground truth information gives you. 
    
    You are not required to do anything more than provide the relevant information as described.
    
    Do NOT give suggestion on how to solve the problem. ONLY categorize it on the information available.
    
    Be sure to NOT give the wrong classification.

    User's problem/situation: {question}
    """
    law_template = PromptTemplate(
        template=law_type_template, input_variables=["context", "question"]
    )
    
    #Actual LLM chain that does classification (RetrievalQA)
    
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=law_fields_vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": law_template},
        #output_key="description"
    ) 
    
    #law_type = chain.run()
    
    law_type = qa.run(query).lower()
    
    return law_type 

The next code cell is a sample query used throughout the notebook for testing. It is very basic and also vague which serves as a good test for proofs of concept.

In [20]:
query = '''
I recieved spoiled milk and I got very sick. The provider is not accepting responsibility.
'''
##Exception case: What if i had the milk later and want to still blame the provider- convo to get this info possible also more
#selective prompts to mention exception cases

In [ ]:
#criminal law- indian penal code , criminal procedure code 

The CriminalLaw() function is not tested. As the query for tests is related to consumer law, that is the kind of law helper which is first being developed. Criminal Law is next followed by the other 16. It will utilize the Indian Penal Code and Criminal Procedure Code.

In [21]:
def CriminalLaw(query):
    
    #prompt for defining nature of situation- formal name for type of felony/crime/injustice
    
    crimina_law_template = """
    You are an assistant who is an expert in understanding law and interpreting language related to law. The user has a problem
    which they describe in a situation. You are to use the given document information context that includes information on
    criminal law and it's various aspects in detail, and using that determine the type of felony or crime or injustice that is 
    in play in the user given situation.
    
    The information context: {context}

    Do NOT assume more information than what the situation and ground truth information gives you. 
    
    You are not required to do anything more than provide the relevant information as described.
    
    Do NOT give suggestion on how to solve the problem. ONLY categorize it on the information available.
    
    Be sure to NOT give the wrong classification.

    User's problem/situation: {question}
    """
    criminal_law_template = PromptTemplate(
        template=criminal_type_template, input_variables=["context", "question"]
    )
    
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=consumercourt_vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": law_type_template},
        #output_key="description"
    ) 
    
    
    #Actual LLM chain that does output generation (RetrievalQA) (IPC)
    
    #Description = chain.run()
    
    #prompt for using description to produce relevant procedures that might be required (RetrievalQA) (CrPC)
    
    #Procedure = chain.run()
    
    #info = Description + Procedure 
    
    return info 

The ConsumerLaw() function inputs the user query and uses a RetrievalQA chain to match it against all the laws in the consumer protection act. 

The goal is to measure the llm's uncertainty level and if it is too uncertain in matching with any law, then it will ask the user a question to clarify that uncertainty.

This way we are able to extract more relevant factual information from the user which allows us to make better classifications. 

The approach used to measure the llm's uncertainty is the "Top-K" method as mentioned in _cite paper here_:

1. The llm is asked to make its top-k (here k=3) predictions, and along with that assign a confidence level which is a number from 0-100 for each law (a percentage). This kind of prompt makes the llm "consider" various options as compared to just being asked for their top prediction and confidence level as then the model becomes overconfident. 

2. The llm is then asked to explain why the confidence level for the highest rated law is not even higher. This gives us an output that explains a source of uncertainty which we can use to ask the user questions and get more clarity.  

In [22]:
def ConsumerLaw(query):
    
    #prompt for defining nature of situation- formal name for type of felony/crime/injustice 
    consumer_law_template = """
    You are an assistant who is an expert in understanding law and interpreting language related to law. The user has a problem
    which they describe in a situation. 
        
    You are to give the top 3 laws from the context which you think apply to this situation. Consider the WHOLE context as best u can. Cite the particular law as per the document context given
    each accompanied by a corresponding probability indicating your confidence level for that guess(expressed as a percentage from 0% to 100%).
    And they MUST SUM UP TO 100%. IT SHOULD NOT SUM UP TO OVER 100%
    
    THE LAW CANNOT BE AN ENTIRE SECTION, BUT MUST BE OF THE FORMAT: SECTION X(number)(letter) to increase specificity.
    
    FOR ONLY THE PARTICULAR LAW WITH HIGHEST CONFIDENCE, EXPLAIN WHY YOUR CONFIDENCE IS NOT EVEN HIGHER FOR IT. IDENTIFY IN PARTICULAR
    POINTS OF AMBIGUITY WHICH CREATE THE CONFUSION AND EXPLAIN THOSE POINTS. Do not just say there may be additional factors and circumstances
    or such, MENTION SPECIFICALLY SUCH FACTORS AND CIRCUMSTANCES. Do not explain the other less confident laws, only the most confident one.
    
    REMEMBER EXPLAIN ONLY THE LAW FOR WHICH YOU HAVE THE HIGHEST CONFIDENCE PERCENTAGE.
    
    THE IDEA IS FOR YOU TO MEASURE YOUR UNCERTAINTY AND IDENTIFY THE AMBIGUITY, AND USE THAT TO ASK THE USER QUESTIONS.
    
    The format must be:
    "Law citation: Percentage" 
    
    The document information context: {context}

    Do NOT assume more information than what the situation and document information context gives you. 
    
    You are not required to do anything more than provide the relevant information as described.
    
    User's problem/situation: {question}
    """
    law_template = PromptTemplate(
        template=consumer_law_template, input_variables=["context", "question"]
    )
    
    #Actual LLM chain that does output generation (RetrievalQA)
    
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=consumer_protection_act_vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": law_template},
        return_source_documents = False,
        #output_key="description"
    )     
    
    description = qa(query)   #query, result, source_document
    
    return description
    

The QuestionGenerator() function takes in the previous llm output from ConsumerLaw() and formulates a question for the user from it. In the prompt template the word "agent" is used. This DOES NOT refer to langchain agents but just actors that perform tasks which is used to depict to this LLM it's specific role in the overall machinery. 

In [23]:
def QuestionGenerator(llm_output):
    

    
    template = """
    There is a three step process to serve the user:
    Agent 1 creates analysis of a situation. 
    Agent 2 frames a question from this analysis to ask the user. YOU ARE AGENT 2.
    The user answers the question
    
    You are an analyst who can understand issues well and is an expert in law.
    
    You are given three laws along with confidence percentage for each, and explains why their confidence is not even higher for it. 
    Your goal is to phrase a question, to ask the user, that if answered, will help in solving the issue mentioned as per the explanation.
    Remember that the user is not an expert in law, so phrase your question in a simple way while still achieving your goal.
    Avoid using complicated terms, and instead use more simpler words if needed. Your question must be as direct and specific as possible.
    
    Note: Address the user/human directly as "you" as if you were in conversation with them. The consumer in this situation is also the user.
    
    Note: Always make sure the question is within ""
    
    Note: Do not mention any of what agent 1 says, just frame a direct question for the user. 
    
    input: {input}
    """
    
    prompt = PromptTemplate(
        template = template,
        input_variables = ['input'],
        #partial_variables = {"da_info":da_info}
    )
    
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    final_output = llm_chain.run(str(llm_output))
                                 
    return final_output
    

The ChainSelector() function is used to connect the LawType() function and the other law field specific functions like the ConsumerLaw() and CriminalLaw() functions.

In [24]:
#IF ELSE BLOCK THAT PUTS INTO DIFF CHAINS? 

def ChainSelector(law_type,query):
     
    
    if(law_type == "criminal law"):
        
        description = CriminalLaw(query)
        
    elif(law_type == "consumer protection law"):

        description = ConsumerLaw(query)
        source_documents = "ConsumerCourt PDF from https://consumerhelpline.gov.in/ConsumerCourt.pdf" #change to consumer protection act 2019 
        
    else:
        
        description = "The law type is: " + str(law_type)
        
    #print("Source document is: " + source_documents)
    #print("")
    #sources = []
    #or doc in description['source_documents']:
        #sources.append(doc.page_content)
    return description, source_documents

The QuestionClassifier() function inputs the question created by the QuestionGenerator() function and decides whether it can be answered by a LLMChain or whether it needs information from the user. So questions like "is milk a product that needs to be handled by experts" should ideally be answered by the llmchain and questions like "where did you get the milk from" should be answered by the user. This allows us to extract more information from a situation and determine which law applies best.

In [25]:
#InfoSourceSelector

def QuestionClassifier(question):
    
    #simple LLMChain that decides between search type and user query type
    
    template = """
    Given to you as input is a question, or a sentence/paragraph that is questioning in nature.
    
    You are an expert in understanding language and requirements. Your goal is to analyse the given input
    and to classify it into either the "search type" or the "user query type".
    
    search type: Deals with theoretical concerns in a situation. Like legal responsibility/liability.These kinds of questions
    tend to be factual in nature and inquire about the natural characteristics, like how products, materials, laws behave, what
    are the definitions of terms, whether certain entities belong to certain classifications. 
    
    user query type: Deals with the practical details relating to actions and events. Questions that specific to a particular situation faced by a person and can therefore ONLY
    be answered by them. Deals with ownership/origin of entities in the situation. 
    
    You are to output either "search type" OR "user query type" based on whichever classification you think is right.
        
    input: {input}
    """
    
    prompt = PromptTemplate(
        template = template,
        input_variables = ['input'],
    )
    
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    final_output = str(llm_chain.run(question))
    details = final_output
    
    #if search type
    #if  final_output == "search type":
    if "search type" in final_output:
        #searchChain
        
        template = """
        Based on the situation given in the input, identify the core issue behind it, and phrase a simple question,
        that if performed a google search upon, will result in facts that answer the question. To this end identify things
        like the core entity/object/relationship and phrase your question around those. Output the question.
        ALSO ANSWER YOUR QUESTION. GIVE A DIRECT ANSWER. 
        input: {input}
        """    
        prompt = PromptTemplate(
            template = template,
            input_variables = ['input'],
        )
    
        llm_chain = LLMChain(llm=llm, prompt=prompt)
    
        question = llm_chain.run(question)    
        
        
        
        sections = question.split("Answer:")
        details = sections[1].strip()
        #print("Answer:", details)
       
    #else if user query type
    elif final_output == "user query type":
        print(question)
        details = input("Enter your response: ")
    
    return details

The following code cell was used to test the QuestionClassifier() function.

In [ ]:
'''
question1 = "Did you purchase the milk yourself or was it provided by your employer for use at the workplace?"

question2 = "According to one of the laws that apply to this situation, if the spoiled milk was a product that required expert handling or supervision, and the manufacturer provided adequate warnings or instructions to experts, they may not be held responsible for any harm caused. Can you provide any information about whether the spoiled milk falls under the category of a product that is legally meant to be used or dispensed only by or under the supervision of an expert or a class of experts, and whether the manufacturer provided warnings or instructions to such experts?"

question3 = 

details = QuestionClassifier(question1)
'''

Finally, the first part of the code is done- we can input a situation from a user and ask them more questions to make specific inferences to see which law is involved. Right now the loop is set to repeat 3 times. This will be changed later to repeat until a confidence of 80+ or 90+ is achieved. Also in future updates both the question and the answer shiuld be added to the query or else it wont make sense as seen in the below outputs.

In [26]:
def FeedbackLoop():
    
    query = input("Enter your situation: ")
    count = 0
    while(count != 3):
        law = LawType(query)  
        analysis, source_documents = ChainSelector(law,query)
        question = QuestionGenerator(analysis)
        details = QuestionClassifier(question)
        query = query + f"\n additional details: {details}" #direct input into query method- also try partial variable method?
        count = count + 1 #whichever happens first- 100% or count of 3 
    return query, analysis  #gotta implement another chain selector man

query, analysis = FeedbackLoop()

Enter your situation:  I recieved spoiled milk and I got very sick. The provider is not accepting responsibility.


"Did you purchase the spoiled milk for use at your workplace?"


Enter your response:  no I bought it for personal use.


"To gather more information and clarify the situation, can you please confirm whether you consumed the milk after its expiration date?"


Enter your response:  No I did not.


In [28]:
print(query)  #add the question to details too? as otherwise it doesnt add value to the end query


I recieved spoiled milk and I got very sick. The provider is not accepting responsibility.
 additional details: The question is asking for confirmation on whether the employer purchased the spoiled milk for use at the workplace and if the manufacturer provided any warnings or instructions regarding the milk.
 additional details: no I bought it for personal use.
 additional details: No I did not.


In [27]:
print(analysis)

{'query': 'I recieved spoiled milk and I got very sick. The provider is not accepting responsibility.\n additional details: The question is asking for confirmation on whether the employer purchased the spoiled milk for use at the workplace and if the manufacturer provided any warnings or instructions regarding the milk.\n additional details: no I bought it for personal use.', 'result': "Based on the given information, the top 3 laws that may apply to this situation are:\n\n1. Section 86: Liability of product sellers - 70%\n2. Section 32(b): Act of omission or commission or negligence - 20%\n3. Section 2(a): Failure to provide adequate warnings or instructions - 10%\n\nExplanation for the law with the highest confidence level (Section 86: Liability of product sellers - 70%):\n\nThe law with the highest confidence level is Section 86, which states that a product seller who is not a product manufacturer shall be liable in a product liability action. In this case, since the user received s

Now we set up a custom prompt agent. 

In [29]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [30]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

SolutionGenerator() is a custom agent made with a custom prompt that is still not working exactly as intended. Its goal is to use all the information accumulated until now and use google searches to find specific solutions and procedures to help the user get justice. This agent requires more fixes.

In [31]:
def SolutionGenerator(query, law_type, description): #, sources, source_documents):

    search = SerpAPIWrapper(serpapi_api_key='')
    tools = [
        Tool(
            name="Intermediate Answer",
            func=search.run,
            description='google search'
        )
    ]

    template = """
    
    You are an assistant who is an expert in understanding law and interpreting language related to law. The user has a problem
    which they describe in a situation. The description of the situation has already been given to you, as well as the type of law involved.
    
    Type of law: {law_type}
    
    Description:{description}

    Assume the user is based in India.
    
    Complete the objective as best you can. You have access to the following tools:

    {tools}

    Use the following format:

    Question: the input question you must answer
        
    Thought: you should always think about what to do.
    Action: the action to take, must be one of [{tool_names}].ALWAYS MAKE AN ACTION. 
    Action Input: the input to the action
    Observation: the result of the action. 
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question. 

    
    Begin!

    Question: {input} based on the summary given under the description.
    {agent_scratchpad}"""
    
    prompt = CustomPromptTemplate(
        template=template,
        tools=tools,
        # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
        # This includes the `intermediate_steps` variable because that is needed
        input_variables=["input", "intermediate_steps", "law_type", "description"],
        #partial_variables={"law_type":law_type, "description":description}
    )
    
    tool_names = [tool.name for tool in tools]
    
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    agent = LLMSingleActionAgent(
        llm_chain=llm_chain,
        output_parser=output_parser,
        stop=["\nObservation:"],
        allowed_tools=tool_names,
    )
    
    agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

    response = agent_executor({"input":query, "law_type":law_type, "description":description})
    
    '''
    log_values = []
    for action in response["intermediate_steps"]:
        log_values.append(action[2])
    
    logs = str(log_values)
    
    template = """
    The given materials is {logs} {placeholder}
    Compile and make an exhaustive summary of the facts and observations of the logs might understand all the relevant information"
    """
    
    prompt = PromptTemplate(
        template = template,
        input_variables = ['placeholder'],
        partial_variables = {"logs":logs}
    )
    
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    response = llm_chain.run("do")
    '''
    return response 

In [33]:
response = SolutionGenerator("where/who do I go to and what do I need for my case to get justice", "consumer protection law", analysis) #, sources, source_documents)#put a while until it takes action 



> Entering new AgentExecutor chain...
Thought: To determine where the user should go and what they need for their case, we need to consider the laws and legal procedures related to consumer protection in India.

Action: Intermediate Answer
Action Input: "consumer protection laws and procedures in India"

Observation:Procedure to File Consumer Case Any consumer complaint relating to a good or service must be filed in writing with a District Forum by the consumer along with the fee. On receipt of a complaint, the District Forum may reject or approve the complaint, usually within 21 days from the date of complaint.
Based on the observation, it seems that the user should file a consumer complaint with the District Forum in India. 

Action: Intermediate Answer
Action Input: "how to file a consumer complaint in India"

Observation:What is the procedure to file a complaint in the Consumer Court?
Based on the observation, it seems that the user should follow the procedure to file a complaint

In [34]:
print(response)
print()
#print(source)
print()
#print(source_documents)

{'input': 'where/who do I go to and what do I need for my case to get justice', 'law_type': 'consumer protection law', 'description': {'query': 'I recieved spoiled milk and I got very sick. The provider is not accepting responsibility.\n additional details: The question is asking for confirmation on whether the employer purchased the spoiled milk for use at the workplace and if the manufacturer provided any warnings or instructions regarding the milk.\n additional details: no I bought it for personal use.', 'result': "Based on the given information, the top 3 laws that may apply to this situation are:\n\n1. Section 86: Liability of product sellers - 70%\n2. Section 32(b): Act of omission or commission or negligence - 20%\n3. Section 2(a): Failure to provide adequate warnings or instructions - 10%\n\nExplanation for the law with the highest confidence level (Section 86: Liability of product sellers - 70%):\n\nThe law with the highest confidence level is Section 86, which states that a p

We extract the "intermediate steps" from the agent. The way the agent works is by asking itself questions and answering them by using tools until it gets an answer that matches the final expected answer. At this point the agent will only output the final answer and will lose all the information and context it gained in its intermediate self questioning steps. Our actual interest is in these steps. So we extract the intermediate steps and parse them to extract the relevant information.

In [35]:
final_info = []
for action in range(len(response["intermediate_steps"])):
    final_info.append("Thoughts: " + str(response["intermediate_steps"][action][0].log))
    final_info.append("Observations: " + str(response["intermediate_steps"][action][1]))

In [36]:
print(final_info)

['Thoughts: Thought: To determine where the user should go and what they need for their case, we need to consider the laws and legal procedures related to consumer protection in India.\n\nAction: Intermediate Answer\nAction Input: "consumer protection laws and procedures in India"', 'Observations: Procedure to File Consumer Case Any consumer complaint relating to a good or service must be filed in writing with a District Forum by the consumer along with the fee. On receipt of a complaint, the District Forum may reject or approve the complaint, usually within 21 days from the date of complaint.', 'Thoughts: Based on the observation, it seems that the user should file a consumer complaint with the District Forum in India. \n\nAction: Intermediate Answer\nAction Input: "how to file a consumer complaint in India"', 'Observations: What is the procedure to file a complaint in the Consumer Court?', 'Thoughts: Based on the observation, it seems that the user should follow the procedure to file

At last, we use the information gained from the agent and create a list of information that can directly help the user. Right now the functionality is extremely basic but I hope to expand and upscale it over time.

In [42]:
    template = """
    The given materials is {final_info} 
    Compile and make an exhaustive summary of the facts and observations of the logs might understand all the relevant information
    Use the information contained in the thoughts and observations, dont say "thoughts" or "observation"
    """
    
    prompt = PromptTemplate(
        template = template,
        input_variables = ['final_info'],
        #partial_variables = {"da_info":da_info}
    )
    
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    final_output = llm_chain.run(str(final_info))

In [43]:
print(final_output)

- The user needs to determine where they should go and what they need for their case related to consumer protection in India.
- The procedure to file a consumer case in India requires the consumer to submit a written complaint with the District Forum along with the fee.
- The District Forum may approve or reject the complaint within 21 days from the date of submission.
- The user should file a consumer complaint with the District Forum in India.
- The user should follow the procedure to file a complaint in the Consumer Court in India.
- The user can refer to an article titled "How to File a Case in Consumer Court?" from Surendranath Evening College for guidance on filing a complaint in the Consumer Court in India.
- The Consumer Protection Act of 1986 provides protection to consumers against unfair trade practices by businesses.
- The act was passed in response to numerous complaints regarding unfair trade practices that resulted in financial and time losses for customers.


All code below this point is extra code that I didnt delete yet as I thought it may be useful later.

In [44]:
def chain_selector(law_type):
    
    if(law_type == "criminal law"):
        #def 
    else if(law_type == "civil law"):
        
    else if(law_type == "family law"):
    
    else if(law_type == "property law"):
    
    else if(law_type == "constitutional law"):
    
    else if(law_type == "administrative law"):
        
    else if(law_type == "tax law"):
        
    else if(law_type == "labor and employment law"):
        
    else if(law_type == "intellectual property law"):
        
    else if(law_type == "cyber law"):
        
    else if(law_type == "international law"):
        
    else if(law_type == "human rights law"):
        
    else if(law_type == "consumer protection law"):
        
    else if(law_type == "tribal and indigenous law"):
        
    else if(law_type == "customary law"):
        
    else if(law_type == "banking and finance law"):
        

IndentationError: expected an indented block after 'if' statement on line 3 (2397810302.py, line 5)

In [ ]:
query = '''
This is my late grandad's house. My uncle rented it out to a family soon after my grandad's death in 2020. Since last year, we've been trying to get them to vacate since we want to sell the property. However, they kept delaying giving various reasons, and we graciously agreed thinking that delaying by a couple of months each time was no biggie since they promised that they would eventually relocate.



Last time (in March of this year), they told us to give them time until Sept. And so, my uncle went to speak to them today. They are flat-out refusing to move out. They said they never planning on leaving the property. They also haven't been paying rent since March because they told us to deduct it from the deposit (my uncle agreed to it; it was foolish). Now the deposit has run out and they say they won't pay rent either. Instead, they proposed a really ridiculous alternative.



Here's their deal - They want to pay 20 Lakhs now for the house and draw up an agreement that states that they will buy the property from us in Feb 2024 and that the remaining consideration will be paid in full then.



We think that this is beyond ludicrous considering the fact that this house is located in Indiranagar and given their behaviour so far, they are more likely to wash their hands off in Feb by stating that they paid the agreed sale price of 20L and we will be left with nothing.



They are from Bangalore itself and have a lot of connections and are hell-bent on keeping the property for themselves through hook or crook.
'''


In [ ]:
  You are to use the given document information context that includes information on consumer law and it's various aspects in detail,
    and using that determine the formal name of the type of felony or crime or injustice that is in play in the user given situation.
    

In [ ]:
    template = """
    Given to you are two questions:
    
    question 1: Did you purchase the milk yourself or was it provided by your employer for use at the workplace?
    
    question 2: According to one of the laws that apply to this situation, if the spoiled milk was a product that required expert handling or supervision, and the manufacturer provided adequate warnings or instructions to experts, they may not be held responsible for any harm caused. Can you provide any information about whether the spoiled milk falls under the category of a product that is legally meant to be used or dispensed only by or under the supervision of an expert or a class of experts, and whether the manufacturer provided warnings or instructions to such experts?
    
    input: {input}
    """
    
    prompt = PromptTemplate(
        template = template,
        input_variables = ['input'],
    )
    
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    final_output = llm_chain.run("What is the underlying logical/characteristic difference between the two questions provided that may differentiate them into different types which can be used to differentiate even other questions")

In [ ]:
print(final_output)

In [ ]:
 """
        template = """
        #Based on the situation given in the input, identify the core issue behind it, and phrase a simple question,
        #that if performed a google search upon, will result in facts that answer the question. To this end identify things
        #like the core entity/object/relationship and phrase your question around those. Output the question.
        #ALSO ANSWER YOUR QUESTION. GIVE A DIRECT ANSWER.
        #input: {input}
        """    
        prompt = PromptTemplate(
            template = template,
            input_variables = ['input'],
        )
    
        llm_chain = LLMChain(llm=llm, prompt=prompt)
    
        question = llm_chain.run(question)    
        
        print(question)
        
        search = SerpAPIWrapper(serpapi_api_key='')
        tools = [
        Tool(
            name="Intermediate Answer",
            func=search.run,
            description='google search'
        )
        ]
        self_ask_with_search = initialize_agent(
        tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
        )
        
        details = self_ask_with_search.run(question)
        #details = (search result)
        """